In [1]:
import os
import os.path as osp
import sys
import numpy as np
import torch, torchvision
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from torchsummary import summary

from torchviz import make_dot, make_dot_from_trace

import mmdet
import mmcv
from mmdet.models import build_detector
from mmcv.parallel import MMDataParallel
from mmdet.datasets import get_dataset
from mmdet.datasets import transforms

In [31]:
import tempfile
from mmdet.core.evaluation import coco_utils
from mmdet.core.post_processing.bbox_nms import multiclass_nms
from mmdet.ops.nms import nms_wrapper

In [3]:
cfg = './visdrone/configs/ssd300.py'
cfg = mmcv.Config.fromfile(cfg)
# gt_json = cfg.data.test.ann_file
dataset = get_dataset(cfg.data.test)

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
load_annotations time: 0.1s from data/VisDrone2019-DET/VisDrone2018-DET-val/annotations_val.json


In [4]:
txt_dir = '/tmp/fuckyoudir/'

In [9]:
def txt2det(fid, num_classes=10):
    """
    Args:
        fid: opened file handler.
    """
    lines = fid.readlines()
    lines = [v.strip('\n') for v in lines]
    lines = [v.split(',') for v in lines]
    dets = [[] for _ in range(num_classes)]
    for line in lines:
        x1, y1, w, h, sc, label, trun, occ = line
        label = int(label)
        if label == 0 or label == 11:
            continue
        assert label > 0 and label < 11, 'Bad label'
        x1, y1 = int(x1), int(y1)
        w, h = int(w), int(h)
        score = float(sc)
        x2 = x1 + w
        y2 = y1 + h
        bbox = np.asarray([x1, y1, x2, y2, score], dtype=np.float32)
        dets[label - 1].append(bbox)
    for i in range(len(dets)):
        if len(dets[i]) == 0:
            dets[i] = np.empty([0, 5])
        else:
            dets[i] = np.stack(dets[i], 0)
    return dets

In [10]:
def get_dets(savedir, dataset):
    results = []
    for info in dataset.img_infos:
        img_name = info['filename']
        stem = img_name.split('.')[0].split('/')[1]

        # do merge
        for i in range(10):
            if i == 9:
                exp_name = '{}_{}.txt'.format(stem, i)
                exp_name = os.path.join(txt_dir, exp_name)
                assert osp.exists(exp_name)
                with open(exp_name) as f:
                    result = txt2det(f)

        results.append(result)    

In [19]:
def get_dets_merge(savedir, dataset):
    results = []
    for info in dataset.img_infos:
        img_name = info['filename']
        stem = img_name.split('.')[0].split('/')[1]

        # do merge
        single_results =  []
        for i in range(10):
            exp_name = '{}_{}.txt'.format(stem, i)
            exp_name = os.path.join(txt_dir, exp_name)
            assert osp.exists(exp_name)
            with open(exp_name) as f:
                sin_res = txt2det(f)
            single_results.append(sin_res)
        
        # 
        num_classes = len(single_results[0])
        per_cls_bboxes = [[] for _ in range(num_classes)]  # [ []*10]
        for result in single_results:
            for i, res in enumerate(result):
                per_cls_bboxes[i].append(res)
        for i in range(num_classes):
            per_cls_bboxes[i] = np.concatenate(per_cls_bboxes[i], 0)
            
        results.append(per_cls_bboxes)    
    return results

In [20]:
# results = get_dets(savedir=txt_dir, dataset=dataset)
results = get_dets_merge(savedir=txt_dir, dataset=dataset)

# NMS

In [87]:
def transform_results_by_nms(results, nms_func, iou_thr = 0.5):
    for idx, res in enumerate(results):
        for i, c_res in enumerate(res):
            bb, ind = nms_func(c_res.astype(np.float32), iou_thr)
            res[i] = bb
    results[idx] = res
    return results

In [88]:
import functools
nms_func = functools.partial(nms_wrapper.soft_nms, method='linear', sigma=float(0.5), min_score=float(1e-3))
ress = transform_results_by_nms(results, nms_func)

tf = tempfile.mkstemp(suffix='.json')
js = coco_utils.results2json(dataset, ress, tf[1])
coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)

Loading and preparing results...
DONE (t=3.32s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=53.23s).
Accumulating evaluation results...
DONE (t=1.43s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.209
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.402
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.343
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.095
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.278
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

{'Precision/mAP': 0.20873067417324626,
 'Precision/mAP_.50IOU': 0.4020375384369735,
 'Precision/mAP_.75IOU': 0.19182219241796314,
 'Precision/mAP_small': 0.14018415315078117,
 'Precision/mAP_medium': 0.2866588372494439,
 'Precision/mAP_large': 0.34259447416985084,
 'Recall/AR_1': 0.09473586379582971,
 'Recall/AR_10': 0.2781087474717202,
 'Recall/AR_100': 0.35552202496517027,
 'Recall/AR_100_small': 0.28237107083780083,
 'Recall/AR_100_medium': 0.44134314886344034,
 'Recall/AR_100_large': 0.4672847293114858}

In [51]:
np.linspace(0.2, 0.95, 10)

array([0.2       , 0.28333333, 0.36666667, 0.45      , 0.53333333,
       0.61666667, 0.7       , 0.78333333, 0.86666667, 0.95      ])

In [52]:
cocores = []
for ithr in np.linspace(0.2, 0.95, 10):
    ress = transform_results_by_nms(results, iou_thr=ithr)

    tf = tempfile.mkstemp(suffix='.json')
    js = coco_utils.results2json(dataset, ress, tf[1])
    cocores_ = coco_utils.coco_eval(tf[1], ['bbox'], dataset.coco)
    cocores.append(cocores_)

Loading and preparing results...
DONE (t=3.32s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=52.93s).
Accumulating evaluation results...
DONE (t=1.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.209
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.402
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.343
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.095
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.278
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

Loading and preparing results...
DONE (t=3.17s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=52.27s).
Accumulating evaluation results...
DONE (t=1.32s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.209
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.402
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.192
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.343
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.095
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.278
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [56]:
[v.shape for v in ress[0]]

[(1495, 5),
 (30, 5),
 (49, 5),
 (160, 5),
 (6, 5),
 (13, 5),
 (3, 5),
 (2, 5),
 (0, 5),
 (32, 5)]